### From Phases to Pixels: Exploring Neural Networks for Image Reconstruction from Phase Spectra
Made by:  
 - Artur Oleksiński
 - Szymon Siąkała

In [ ]:
import contextlib
import numpy as np
import os

from matplotlib import pyplot
from model import define_generator, define_discriminator, define_composite_model, train
from preprocessing import (
    rename_images,
    convert_to_grayscale,
    generate_phases_and_magnitudes,
    load_images,
    load_phases_and_magnitudes,
    print_sample,
)
from translation import load_image_phase_magnitude, load_custom_model, predict, prepare_magnitude_or_phase

In [ ]:
def preprocessing(color_path, grayscale_path, magnitudes_path, phases_path):
    """
    Preprocess the images to be used in the model

    :param color_path: Path to the folder containing the color images
    :param grayscale_path: Path to the folder where the grayscale images will be saved
    :param magnitudes_path: Path to the folder where the magnitudes will be saved
    :param phases_path: Path to the folder where the phases will be saved
    """

    rename_images(color_path)
    convert_to_grayscale(color_path, grayscale_path)
    generate_phases_and_magnitudes(grayscale_path, phases_path, magnitudes_path)


def load_dataset(grayscale_path, magnitudes_path, phases_path, num_images):
    """
    Load the dataset to be used in the model

    :param grayscale_path: Path to the folder containing the grayscale images
    :param magnitudes_path: Path to the folder containing the magnitudes
    :param phases_path: Path to the folder containing the phases
    :param num_images: Number of images to load
    :return: Tuple containing the grayscale images and the magnitudes and phases
    """

    grayscale_images = load_images(grayscale_path, num_images)
    magnitudes_and_phases = load_phases_and_magnitudes(phases_path, magnitudes_path, num_images)

    return grayscale_images, magnitudes_and_phases


def train_model(grayscale_images, phases, epochs, batch_size):
    """
    Train the model

    :param grayscale_images: Grayscale images
    :param magnitudes_and_phases: Magnitudes and phases
    """

    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        image_shape = grayscale_images.shape[1:]
        g_model_AtoB = define_generator(image_shape)
        g_model_BtoA = define_generator(image_shape)
        d_model_A = define_discriminator(image_shape)
        d_model_B = define_discriminator(image_shape)
        c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
        c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

        train(
            d_model_A,
            d_model_B,
            g_model_AtoB,
            g_model_BtoA,
            c_model_AtoB,
            c_model_BtoA,
            [grayscale_images, phases],
            epochs=epochs,
            batch=batch_size,
        )


def predict_with_model(moodel_path, phase):
    """
    Predict an image using a custom model

    :param moodel_path: Path to the model
    :param phase: Phase
    :return: The predicted image
    """

    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        model_BtoA = load_custom_model(moodel_path)
        model_input = prepare_magnitude_or_phase(phase)

        return predict(model_BtoA, model_input)


def print_results(grayscale, phase, prediction):
    """
    Print the results of the model

    :param grayscale: Grayscale image
    :param magnitude: Magnitude
    :param phase: Phase
    :param generated: Generated image
    """

    fig = pyplot.figure()

    pyplot.subplot(1, 3, 1)
    pyplot.axis("off")
    pyplot.imshow(grayscale.astype("uint8"))
    pyplot.title("Original Image")

    pyplot.subplot(1, 3, 2)
    pyplot.axis("off")
    pyplot.imshow(phase)
    pyplot.title("Phase")

    pyplot.subplot(1, 3, 3)
    pyplot.axis("off")
    pyplot.imshow(prediction)
    pyplot.title("Predicted Image")

    mse = np.mean((grayscale - prediction) ** 2)
    fig.text(0.5, 0.3, f"MSE: {mse:.2f}", ha="center", fontsize=12)
    pyplot.show()

In [ ]:
# Parameters
color_path = "../data/color"
grayscale_path = "../data/grayscale"
magnitudes_path = "../data/magnitudes"
phases_path = "../data/phases"
moodel_path = "../data/models/s200e50/g_model_BtoA_002500.h5"
num_images = 5
epochs = 5
batch = 1
numbers_of_images_to_predict = [420, 666, 911, 2137, 4321]
testing_images = 500

In [ ]:
# Preprocessing
preprocessing(color_path, grayscale_path, magnitudes_path, phases_path)
grayscale_images, magnitudes_and_phases = load_dataset(grayscale_path, magnitudes_path, phases_path, num_images)
print_sample(grayscale_images, magnitudes_and_phases, 5)

In [ ]:
# Model training
phases = np.asarray([phase for _, phase in magnitudes_and_phases])
phases = np.repeat(phases[:, :, :, np.newaxis], 3, axis=3)
train_model(grayscale_images, phases, epochs, batch)

In [ ]:
# Creating predictions
for num in numbers_of_images_to_predict:
    grayscale, _, phase = load_image_phase_magnitude(grayscale_path, magnitudes_path, phases_path, num)
    prediction = predict_with_model(moodel_path, phase)
    print_results(grayscale, phase, prediction)

In [ ]:
# Model testing
mse_array = []
model_BtoA = load_custom_model(moodel_path)

with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    for i in range(testing_images):
        grayscale, _, phase = load_image_phase_magnitude(grayscale_path, magnitudes_path, phases_path, i)
        model_input = prepare_magnitude_or_phase(phase)
        prediction = predict(model_BtoA, model_input)
        mse_array.append(np.mean((grayscale - prediction) ** 2))

print(f"Average Mean Squared Error: {np.mean(mse_array)}")